In [1]:
import os
os.chdir('../')

### from "sample noise data pair by Euler and save pt.ipynb"

In [2]:
import torch
import torch.nn.functional as F

save_file = 'optimization/pair/euler_120.pt'
pairs = torch.load(save_file)
noise = pairs[:, 0]
target = pairs[:, 1]
print(noise.shape, target.shape)

torch.Size([1024, 3, 64, 64]) torch.Size([1024, 3, 64, 64])


In [3]:
import sys
import torch
import numpy as np
import matplotlib.pyplot as plt
import torchvision.utils as vutils
import random

from main import parse_args_and_config, Diffusion
from datasets import inverse_data_transform

seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.benchmark = False

###############################################################################
# 1) Notebook에서 sys.argv를 직접 설정 (argparse 흉내)
###############################################################################
sys.argv = [
    "main.py",
    "--config", "imagenet64.yml",  # 사용하려는 config
    "--sample",
    "--sample_type", "rbfsolvernumericalsweep",
    "--dpm_solver_type", "data_prediction",
    "--dpm_solver_order", "3",
    "--timesteps", "5",
    "--skip_type", "logSNR",
    "--log_scale_min", "0.0",
    "--log_scale_max", "6.0",
    "--scale_dir", "/data/data/rbfsolvernumericalsweep_scales",
    "--ni"
]

###############################################################################
# 2) 인자/설정 로드
###############################################################################
args, config = parse_args_and_config()

###############################################################################
# 3) Diffusion 객체 생성 -> 모델 로딩
###############################################################################
diffusion = Diffusion(args, config, rank=0)
diffusion.prepare_model()
diffusion.model.eval()

###############################################################################
# 4) 배치(25장) 한 번에 샘플링 -> 5x5 그리드(여백 없이) 시각화
###############################################################################
device = diffusion.device


INFO - main.py - 2025-03-26 13:04:22,049 - Using device: cuda


[prepare_model] Loading ckpt from /data/checkpoints/dpm-solver/imagenet64_uncond_100M_1500K.pt ...
[prepare_model] Checkpoint loaded.
[prepare_model] Model is ready.


In [4]:
!mkdir /data/data/rbfsolvernumericalsweep_scales

mkdir: cannot create directory ‘/data/data/rbfsolvernumericalsweep_scales’: File exists


In [5]:
n = 256
noise_batch = noise[:n].to(device)
target_batch = target[:n].to(device)
(pred, optimal_gammas), _ = diffusion.sample_image(noise_batch, diffusion.model, target=target_batch)
loss = F.mse_loss(pred, target_batch)
print('loss :', loss)


log_scale_min : 0.0 log_scale_max : 6.0


/home/logpx/scpark/dpm-solver-fid/examples/ddpm_and_guided-diffusion/dpm_solver/rbf_solver_numerical_sweep.py:123: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  val, _ = quad(integrand, float(lambda_s), float(lambda_t))


[tensor(0.0469, device='cuda:0'), tensor(0.0426, device='cuda:0'), tensor(0.0406, device='cuda:0'), tensor(0.0366, device='cuda:0'), tensor(0.0374, device='cuda:0'), tensor(0.0290, device='cuda:0')]
/data/data/rbfsolvernumericalsweep_scales/NFE=5,p=3.npy  saved!
loss : tensor(0.0290, device='cuda:0')


### Batch Mode

In [ ]:
import sys
import torch
import numpy as np
from main import parse_args_and_config, Diffusion
from tqdm import tqdm

import torch
import torch.nn.functional as F

save_file = 'optimization/pair/euler_NFE=120_N=2048.pt'
pairs = torch.load(save_file)
noise = pairs[:, 0]
target = pairs[:, 1]
print(noise.shape, target.shape)

model = None
for NFE in tqdm([5, 6, 8, 10, 12, 15, 20, 25]):
    sys.argv = [
        "main.py",
        "--config", "imagenet64.yml",  # 사용하려는 config
        "--sample",
        "--sample_type", "rbfsolverquadsweep",
        "--dpm_solver_type", "data_prediction",
        "--dpm_solver_order", "3",
        "--timesteps", str(NFE),
        "--skip_type", "logSNR",
        "--log_scale_min", "-6.0",
        "--log_scale_max1", "1.0",
        "--log_scale_max2", "6.0",
        "--log_scale_num", "100",
        "--scale_dir", "/data/data/rbfsolverquadsweep",
        "--exp_num", "0",
        "--verbose", "critical",
        "--ni",
    ]

    args, config = parse_args_and_config()
    diffusion = Diffusion(args, config, rank=0)

    if model is None:
        diffusion.prepare_model()
        diffusion.model.eval()
        model = diffusion.model
    device = diffusion.device

    n = 256
    noise_batch = noise[:n].to(device)
    target_batch = target[:n].to(device)
    (pred, _), _ = diffusion.sample_image(noise_batch, model, target=target_batch)
    print(F.mse_loss(pred, target_batch))


torch.Size([1024, 3, 64, 64]) torch.Size([1024, 3, 64, 64])


  0%|          | 0/8 [00:00<?, ?it/s]

[prepare_model] Loading ckpt from /data/checkpoints/dpm-solver/imagenet64_uncond_100M_1500K.pt ...
[prepare_model] Checkpoint loaded.
[prepare_model] Model is ready.


/home/logpx/scpark/dpm-solver-fid/examples/ddpm_and_guided-diffusion/dpm_solver/rbf_solver_quad_sweep.py:118: IntegrationWarning: The occurrence of roundoff error is detected, which prevents 
  the requested tolerance from being achieved.  The error may be 
  underestimated.
  val, _ = quad(integrand, float(lambda_s), float(lambda_t))
/home/logpx/scpark/dpm-solver-fid/examples/ddpm_and_guided-diffusion/dpm_solver/rbf_solver_quad_sweep.py:118: IntegrationWarning: The maximum number of subdivisions (50) has been achieved.
  If increasing the limit yields no improvement it is advised to analyze 
  the integrand in order to determine the difficulties.  If the position of a 
  local difficulty can be determined (singularity, discontinuity) one will 
  probably gain from splitting up the interval and calling the integrator 
  on the subranges.  Perhaps a special-purpose integrator should be used.
  val, _ = quad(integrand, float(lambda_s), float(lambda_t))
 12%|█▎        | 1/8 [00:41<04:53, 

/data/data/rbfsolverquadsweep/NFE=5,p=3,exp_num=0.npy  saved!
tensor(0.0344, device='cuda:0')


...

 75%|███████▌  | 6/8 [08:11<03:24, 102.24s/it]

/data/data/rbfsolverquadsweep/NFE=15,p=3,exp_num=0.npy  saved!
tensor(0.0027, device='cuda:0')


 75%|███████▌  | 6/8 [11:17<03:45, 112.84s/it]


KeyboardInterrupt: 

: 

In [4]:
print('done')

done
